In [204]:
import numpy as np
from numpy import ndarray

In [205]:
class Layer:
    def __init__(self):
        pass

    def forward(self, inp: ndarray) -> ndarray:
        return inp

    def backward(self, inp: ndarray, grad_outp: ndarray):
        pass

In [206]:
class ReLU(Layer):
    def __init__(self):
        pass

    def forward(self, inp: ndarray) -> ndarray:
        return np.where(inp>0, inp, 0)

    def backward(self, inp: ndarray, grad_outp: ndarray):
        pass

In [207]:
class Sigmoid(Layer):
    def __init__(self):
        pass

    def forward(self, inp: ndarray):
        return 1/(1+np.exp(-inp))

    def backward(self, inp: ndarray, grad_outp: ndarray):
        pass

In [208]:
class Dense(Layer):
    def __init__(self, inp_units: int, outp_units: int, learning_rate: float = 0.1):
        self.w = np.random.normal(size=(inp_units, outp_units))
        self.b = np.zeros(outp_units)
        self.lr = learning_rate

    def forward(self, inp: ndarray) -> ndarray:
        print(inp)
        return np.dot(inp, self.w) + self.b

    def backward(self, inp: ndarray, grad_outp: ndarray):
        pass

In [209]:
class MLP:
    def __init__(self):
        self.layers = []

    # adds a dense layer with the input and output shapes
    def _add_dense(self, out_shape: int, inp_shape: int):
        if inp_shape:
            self.layers.append(Dense(inp_shape, out_shape))
        elif self.layers:
            self.layers.append(Dense(len(self.layers[-2].b), out_shape))
        else:
            raise AttributeError('No input shape!')

    # adds an activation layer
    def _add_activation(self, activation: str):
        if activation.lower() == 'sigmoid':
            self.layers.append(Sigmoid())
        elif activation.lower() == 'relu':
            self.layers.append(ReLU())
        else:
            raise AttributeError('No activation!')

    # combination of two methods above
    def add_layer(self, out_shape: int, inp_shape: int|None = None, activation: str = 'sigmoid'):
        self._add_dense(out_shape, inp_shape)
        self._add_activation(activation)

    # cross entropy error
    def ce_loss(self, y_hat: ndarray, y: ndarray) -> float:
        return -np.sum(y*np.log(y_hat+1e-20))

    # performs forward pass
    def forward(self, X: ndarray) -> ndarray:
        for layer in self.layers:
            X = layer.forward(X)
        return X

    # predicts classes
    def predict(self, X: ndarray) -> ndarray:
        return np.argmax(self.forward(X), axis=0)

    # training method
    def fit(self, X: ndarray, y: ndarray):
        pass

In [210]:
test = [[300, 400, 500], [2, 0, 1]]
test = np.array(test)

network = MLP()

network.add_layer(6, inp_shape=3, activation='relu')
network.add_layer(3)

print(network.predict(test[0]))

[[300 400 500]
 [  2   0   1]]
[[792.65446484   0.         342.9262797  408.65676328   0.
    0.        ]
 [  1.71986041   0.           2.76692293   2.16050773   0.
    0.        ]]
[0 1 1]


C:\Users\Mykhailo\AppData\Local\Temp\ipykernel_7620\373195030.py:6: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-inp))
